In [4]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import keras.backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import cv2

In [5]:
"""
InceptionNeXt implementation, paper: https://arxiv.org/abs/2303.16900

Some code is borrowed from timm: https://github.com/huggingface/pytorch-image-models
"""

from functools import partial

import torch
import torch.nn as nn

from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import checkpoint_seq
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model
from timm.models.layers.helpers import to_2tuple


class InceptionDWConv2d(nn.Module):
    """ Inception depthweise convolution
    """
    def __init__(self, in_channels, square_kernel_size=3, band_kernel_size=11, branch_ratio=0.125):
        super().__init__()
        
        gc = int(in_channels * branch_ratio) # channel numbers of a convolution branch
        self.dwconv_hw = nn.Conv2d(gc, gc, square_kernel_size, padding=square_kernel_size//2, groups=gc)
        self.dwconv_w = nn.Conv2d(gc, gc, kernel_size=(1, band_kernel_size), padding=(0, band_kernel_size//2), groups=gc)
        self.dwconv_h = nn.Conv2d(gc, gc, kernel_size=(band_kernel_size, 1), padding=(band_kernel_size//2, 0), groups=gc)
        self.split_indexes = (in_channels - 3 * gc, gc, gc, gc)
        
    def forward(self, x):
        x_id, x_hw, x_w, x_h = torch.split(x, self.split_indexes, dim=1)
        return torch.cat(
            (x_id, self.dwconv_hw(x_hw), self.dwconv_w(x_w), self.dwconv_h(x_h)), 
            dim=1,
        )


class ConvMlp(nn.Module):
    """ MLP using 1x1 convs that keeps spatial dims
    copied from timm: https://github.com/huggingface/pytorch-image-models/blob/v0.6.11/timm/models/layers/mlp.py
    """
    def __init__(
            self, in_features, hidden_features=None, out_features=None, act_layer=nn.ReLU,
            norm_layer=None, bias=True, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        bias = to_2tuple(bias)

        self.fc1 = nn.Conv2d(in_features, hidden_features, kernel_size=1, bias=bias[0])
        self.norm = norm_layer(hidden_features) if norm_layer else nn.Identity()
        self.act = act_layer()
        self.drop = nn.Dropout(drop)
        self.fc2 = nn.Conv2d(hidden_features, out_features, kernel_size=1, bias=bias[1])

    def forward(self, x):
        x = self.fc1(x)
        x = self.norm(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        return x


class MlpHead(nn.Module):
    """ MLP classification head
    """
    def __init__(self, dim, num_classes=1, mlp_ratio=3, act_layer=nn.GELU,
        norm_layer=partial(nn.LayerNorm, eps=1e-6), drop=0., bias=True):
        super().__init__()
        hidden_features = int(mlp_ratio * dim)
        self.fc1 = nn.Linear(dim, hidden_features, bias=bias)
        self.act = act_layer()
        self.norm = norm_layer(hidden_features)
        self.fc2 = nn.Linear(hidden_features, num_classes, bias=bias)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = x.mean((2, 3)) # global average pooling
        x = self.fc1(x)
        x = self.act(x)
        x = self.norm(x)
        x = self.drop(x)
        x = self.fc2(x)
        return x


class MetaNeXtBlock(nn.Module):
    """ MetaNeXtBlock Block
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        ls_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """

    def __init__(
            self,
            dim,
            token_mixer=nn.Identity,
            norm_layer=nn.BatchNorm2d,
            mlp_layer=ConvMlp,
            mlp_ratio=4,
            act_layer=nn.GELU,
            ls_init_value=1e-6,
            drop_path=0.,
            
    ):
        super().__init__()
        self.token_mixer = token_mixer(dim)
        self.norm = norm_layer(dim)
        self.mlp = mlp_layer(dim, int(mlp_ratio * dim), act_layer=act_layer)
        self.gamma = nn.Parameter(ls_init_value * torch.ones(dim)) if ls_init_value else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        shortcut = x
        x = self.token_mixer(x)
        x = self.norm(x)
        x = self.mlp(x)
        if self.gamma is not None:
            x = x.mul(self.gamma.reshape(1, -1, 1, 1))
        x = self.drop_path(x) + shortcut
        return x


class MetaNeXtStage(nn.Module):
    def __init__(
            self,
            in_chs,
            out_chs,
            ds_stride=2,
            depth=2,
            drop_path_rates=None,
            ls_init_value=1.0,
            token_mixer=nn.Identity,
            act_layer=nn.GELU,
            norm_layer=None,
            mlp_ratio=4,
    ):
        super().__init__()
        self.grad_checkpointing = False
        if ds_stride > 1:
            self.downsample = nn.Sequential(
                norm_layer(in_chs),
                nn.Conv2d(in_chs, out_chs, kernel_size=ds_stride, stride=ds_stride),
            )
        else:
            self.downsample = nn.Identity()

        drop_path_rates = drop_path_rates or [0.] * depth
        stage_blocks = []
        for i in range(depth):
            stage_blocks.append(MetaNeXtBlock(
                dim=out_chs,
                drop_path=drop_path_rates[i],
                ls_init_value=ls_init_value,
                token_mixer=token_mixer,
                act_layer=act_layer,
                norm_layer=norm_layer,
                mlp_ratio=mlp_ratio,
            ))
            in_chs = out_chs
        self.blocks = nn.Sequential(*stage_blocks)

    def forward(self, x):
        x = self.downsample(x)
        if self.grad_checkpointing and not torch.jit.is_scripting():
            x = checkpoint_seq(self.blocks, x)
        else:
            x = self.blocks(x)
        return x


class MetaNeXt(nn.Module):
    r""" MetaNeXt
        A PyTorch impl of : `InceptionNeXt: When Inception Meets ConvNeXt`  - https://arxiv.org/pdf/2203.xxxxx.pdf

    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1
        depths (tuple(int)): Number of blocks at each stage. Default: (3, 3, 9, 3)
        dims (tuple(int)): Feature dimension at each stage. Default: (96, 192, 384, 768)
        token_mixers: Token mixer function. Default: nn.Identity
        norm_layer: Normalziation layer. Default: nn.BatchNorm2d
        act_layer: Activation function for MLP. Default: nn.GELU
        mlp_ratios (int or tuple(int)): MLP ratios. Default: (4, 4, 4, 3)
        head_fn: classifier head
        drop_rate (float): Head dropout rate
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        ls_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """

    def __init__(
            self,
            in_chans=1,
            num_classes=1,
            depths=(3, 3, 9, 3),
            dims=(96, 192, 384, 768),
            token_mixers=nn.Identity,
            norm_layer=nn.BatchNorm2d,
            act_layer=nn.GELU,
            mlp_ratios=(4, 4, 4, 3),
            head_fn=MlpHead,
            drop_rate=0.,
            drop_path_rate=0.,
            ls_init_value=1e-6,
            **kwargs,
    ):
        super().__init__()

        num_stage = len(depths)
        if not isinstance(token_mixers, (list, tuple)):
            token_mixers = [token_mixers] * num_stage
        if not isinstance(mlp_ratios, (list, tuple)):
            mlp_ratios = [mlp_ratios] * num_stage


        self.num_classes = num_classes
        self.drop_rate = drop_rate
        self.stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            norm_layer(dims[0])
        )

        self.stages = nn.Sequential()
        dp_rates = [x.tolist() for x in torch.linspace(0, drop_path_rate, sum(depths)).split(depths)]
        stages = []
        prev_chs = dims[0]
        # feature resolution stages, each consisting of multiple residual blocks
        for i in range(num_stage):
            out_chs = dims[i]
            stages.append(MetaNeXtStage(
                prev_chs,
                out_chs,
                ds_stride=2 if i > 0 else 1, 
                depth=depths[i],
                drop_path_rates=dp_rates[i],
                ls_init_value=ls_init_value,
                act_layer=act_layer,
                token_mixer=token_mixers[i],
                norm_layer=norm_layer,
                mlp_ratio=mlp_ratios[i],
            ))
            prev_chs = out_chs
        self.stages = nn.Sequential(*stages)
        self.num_features = prev_chs
        self.head = head_fn(self.num_features, num_classes, drop=drop_rate)
        self.apply(self._init_weights)

    @torch.jit.ignore
    def set_grad_checkpointing(self, enable=True):
        for s in self.stages:
            s.grad_checkpointing = enable

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'norm'}
    

    def forward_features(self, x):
        x = self.stem(x)
        x = self.stages(x)
        return x

    def forward_head(self, x):
        x = self.head(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.forward_head(x)
        return x


    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)



def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1, 'input_size': (3, 224, 224), 'pool_size': (7, 7),
        'crop_pct': 0.875, 'interpolation': 'bicubic',
        'mean': IMAGENET_DEFAULT_MEAN, 'std': IMAGENET_DEFAULT_STD,
        'first_conv': 'stem.0', 'classifier': 'head.fc',
        **kwargs
    }


default_cfgs = dict(
    inceptionnext_tiny=_cfg(
        url='https://github.com/sail-sg/inceptionnext/releases/download/model/inceptionnext_tiny.pth',
    ),
    inceptionnext_small=_cfg(
        url='https://github.com/sail-sg/inceptionnext/releases/download/model/inceptionnext_small.pth',
    ),
    inceptionnext_base=_cfg(
        url='https://github.com/sail-sg/inceptionnext/releases/download/model/inceptionnext_base.pth',
    ),
    inceptionnext_base_384=_cfg(
        url='https://github.com/sail-sg/inceptionnext/releases/download/model/inceptionnext_base_384.pth',
        input_size=(3, 384, 384), crop_pct=1.0,
    ),
)


@register_model
def inceptionnext_tiny(pretrained=False, **kwargs):
    model = MetaNeXt(depths=(3, 3, 9, 3), dims=(96, 192, 384, 768), 
                      token_mixers=InceptionDWConv2d,
                      **kwargs
    )
    model.default_cfg = default_cfgs['inceptionnext_tiny']
    if pretrained:
        state_dict = torch.hub.load_state_dict_from_url(
            url=model.default_cfg['url'], map_location="cpu", check_hash=True)
        model.load_state_dict(state_dict)
    return model

@register_model
def inceptionnext_small(pretrained=False, **kwargs):
    model = MetaNeXt(depths=(3, 3, 27, 3), dims=(96, 192, 384, 768), 
                      token_mixers=InceptionDWConv2d,
                      **kwargs
    )
    model.default_cfg = default_cfgs['inceptionnext_small']
    if pretrained:
        state_dict = torch.hub.load_state_dict_from_url(
            url=model.default_cfg['url'], map_location="cpu", check_hash=True)
        model.load_state_dict(state_dict)
    return model

@register_model
def inceptionnext_base(pretrained=False, **kwargs):
    model = MetaNeXt(depths=(3, 3, 27, 3), dims=(128, 256, 512, 1024), 
                      token_mixers=InceptionDWConv2d,
                      **kwargs
    )
    model.default_cfg = default_cfgs['inceptionnext_base']
    if pretrained:
        state_dict = torch.hub.load_state_dict_from_url(
            url=model.default_cfg['url'], map_location="cpu", check_hash=True)
        model.load_state_dict(state_dict)
    return model

@register_model
def inceptionnext_base_384(pretrained=False, **kwargs):
    model = MetaNeXt(depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], 
                      mlp_ratios=[4, 4, 4, 3],
                      token_mixers=InceptionDWConv2d,
                      **kwargs
    )
    model.default_cfg = default_cfgs['inceptionnext_base_384']
    if pretrained:
        state_dict = torch.hub.load_state_dict_from_url(
            url=model.default_cfg['url'], map_location="cpu", check_hash=True)
        model.load_state_dict(state_dict)
    return model

C:\Users\ameyd\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
x=inceptionnext_small()

In [5]:
model = inceptionnext_small()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
from PIL import Image
import numpy as np
import torch


def processImage(image):
    # Load the image using Pillow (PIL)
    image = Image.open(image)

    # Resize the image to the desired size
    desired_width = 100
    transform = transforms.Compose([
        transforms.Resize((desired_width, desired_width)),
        transforms.ToTensor(),
    ])

    image_tensor = transform(image)

    # Add a batch dimension by unsqueezing
    image_tensor = image_tensor.unsqueeze(0)

    return image_tensor

In [8]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Define a transformation to resize and normalize your images
# Define a transformation to resize and normalize your grayscale images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust to your model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Mean and std for grayscale images
])

In [8]:
imageData=pd.read_csv('../../data/siim-acr-pneumothorax/stage_1_train_images.csv')

In [9]:
imageData

,new_filename,ImageId,has_pneumo
0,0_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.5597.151787518...,0
1,1_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.12515.15178752...,0
2,2_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.4904.151787518...,1
3,3_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.32579.15178751...,1
4,4_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.1314.151787516...,1
...,...,...,...
10670,10670_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.32459.15178751...,1
10671,10671_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.4461.151787518...,1
10672,10672_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.32730.15178751...,0
10673,10673_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.13252.15178752...,0


In [30]:
type(imageData)

pandas.core.frame.DataFrame

In [10]:
labels = imageData['has_pneumo']

In [11]:
len(labels)

10675

In [12]:
import os

data_dir = '../../data/siim-acr-pneumothorax/png_images'
image_filenames = os.listdir(data_dir)  # List of image file names in your data directory
image_paths = [os.path.join(data_dir, filename) for filename in image_filenames]

# Make sure image_paths contains the correct file paths to your images
#print(image_paths)

In [13]:
image_paths = []

for i in range(len(imageData)):
    file_name = '../../data/siim-acr-pneumothorax/png_images/' + imageData['new_filename'][i]
    image_paths.append(file_name)

In [14]:
len(image_paths)

10675

In [15]:
custom_dataset = CustomDataset(image_paths, labels, transform=transform)

In [16]:
import torch.cuda

torch.cuda.is_available()

False

In [17]:
from pprint import pprint
pprint(vars(custom_dataset))

{'image_paths': ['../../data/siim-acr-pneumothorax/png_images/0_train_0_.png',
                 '../../data/siim-acr-pneumothorax/png_images/1_train_0_.png',
                 '../../data/siim-acr-pneumothorax/png_images/2_train_1_.png',
                 '../../data/siim-acr-pneumothorax/png_images/3_train_1_.png',
                 '../../data/siim-acr-pneumothorax/png_images/4_train_1_.png',
                 '../../data/siim-acr-pneumothorax/png_images/5_train_0_.png',
                 '../../data/siim-acr-pneumothorax/png_images/6_train_0_.png',
                 '../../data/siim-acr-pneumothorax/png_images/7_train_1_.png',
                 '../../data/siim-acr-pneumothorax/png_images/8_train_1_.png',
                 '../../data/siim-acr-pneumothorax/png_images/9_train_0_.png',
                 '../../data/siim-acr-pneumothorax/png_images/10_train_1_.png',
                 '../../data/siim-acr-pneumothorax/png_images/11_train_1_.png',
                 '../../data/siim-acr-pneumothorax

In [18]:
train_loader = DataLoader(custom_dataset, batch_size=100, shuffle=True)

In [19]:
train_loader

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model
x = inceptionnext_small()

# Define a binary cross-entropy loss function
criterion = nn.BCELoss()

# Define an optimizer (e.g., Adam)
optimizer = optim.Adam(x.parameters(), lr=0.1)

# Training loop
num_epochs = 1  # Adjust the number of epochs as needed

current_loss = 0
loss_count = 0

for epoch in range(num_epochs):
    x.train()
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = x(images)
        outputs = torch.sigmoid(outputs)  # Apply sigmoid activation
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    
    if current_loss < average_loss+average_loss*0.05 or current_loss > average_loss-average_loss*0.05:
        loss_count += 1
        current_loss = average_loss
    
    if loss_count > 3:
        break
        
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {average_loss:.4f}")

# Save the trained model if needed
torch.save(x.state_dict(), 'inceptionnext_small_trained.pth')

Epoch [1/1] Loss: 21.9885


In [ ]:
#Testing the model

In [9]:
imageData=pd.read_csv('../../data/siim-acr-pneumothorax/stage_1_test_images.csv')

In [10]:
imageData

,new_filename,ImageId,has_pneumo
0,0_test_1_.png,1.2.276.0.7230010.3.1.4.8323329.5797.151787519...,1
1,1_test_0_.png,1.2.276.0.7230010.3.1.4.8323329.5798.151787519...,0
2,2_test_0_.png,1.2.276.0.7230010.3.1.4.8323329.5799.151787519...,0
3,3_test_0_.png,1.2.276.0.7230010.3.1.4.8323329.580.1517875163...,0
4,4_test_0_.png,1.2.276.0.7230010.3.1.4.8323329.5800.151787519...,0
...,...,...,...
1367,1367_test_0_.png,1.2.276.0.7230010.3.1.4.8323329.7049.151787520...,0
1368,1368_test_1_.png,1.2.276.0.7230010.3.1.4.8323329.705.1517875164...,1
1369,1369_test_0_.png,1.2.276.0.7230010.3.1.4.8323329.7050.151787520...,0
1370,1370_test_0_.png,1.2.276.0.7230010.3.1.4.8323329.7051.151787520...,0


In [11]:
labels = imageData['has_pneumo']

In [12]:
image_paths = []

for i in range(len(imageData)):
    file_name = '../../data/siim-acr-pneumothorax/png_images/' + imageData['new_filename'][i]
    image_paths.append(file_name)

In [13]:
custom_dataset = CustomDataset(image_paths, labels, transform=transform)

In [14]:
test_loader = DataLoader(custom_dataset, batch_size=100, shuffle=True)

In [15]:
x.eval()

MetaNeXt(
  (stem): Sequential(
    (0): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (stages): Sequential(
    (0): MetaNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): MetaNeXtBlock(
          (token_mixer): InceptionDWConv2d(
            (dwconv_hw): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=12)
            (dwconv_w): Conv2d(12, 12, kernel_size=(1, 11), stride=(1, 1), padding=(0, 5), groups=12)
            (dwconv_h): Conv2d(12, 12, kernel_size=(11, 1), stride=(1, 1), padding=(5, 0), groups=12)
          )
          (norm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (mlp): ConvMlp(
            (fc1): Conv2d(96, 384, kernel_size=(1, 1), stride=(1, 1))
            (norm): Identity()
            (act): GELU(approximate='none')
            (drop): Dropout(p=0.0, inplace=False)
 

In [16]:
correct_predictions = 0
total_samples = 0

In [32]:
with torch.no_grad():
    for images, labels in test_loader:
        #images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = x(images)
        predicted = (outputs > 0.5).float()  # Assuming binary classification
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
accuracy = correct_predictions / total_samples
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 77.6589


In [1]:
from sklearn.metrics import classification_report

y_true = [label.item() for label in labels]  # Convert labels to a list
y_pred = [predicted.item() for pred in predicted]  # Convert predicted labels to a list

report = classification_report(y_true, y_pred)
print(report)

NameError: name 'labels' is not defined

MetaNeXt(
  (stem): Sequential(
    (0): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (stages): Sequential(
    (0): MetaNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): MetaNeXtBlock(
          (token_mixer): InceptionDWConv2d(
            (dwconv_hw): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=12)
            (dwconv_w): Conv2d(12, 12, kernel_size=(1, 11), stride=(1, 1), padding=(0, 5), groups=12)
            (dwconv_h): Conv2d(12, 12, kernel_size=(11, 1), stride=(1, 1), padding=(5, 0), groups=12)
          )
          (norm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (mlp): ConvMlp(
            (fc1): Conv2d(96, 384, kernel_size=(1, 1), stride=(1, 1))
            (norm): Identity()
            (act): GELU(approximate='none')
            (drop): Dropout(p=0.0, inplace=False)
 

In [19]:
precision

0.0

In [20]:
f1

0.0

In [23]:
model=inceptionnext_small()
model.load_state_dict(torch.load('inceptionnext_small_trained.pth'))
model.eval()

MetaNeXt(
  (stem): Sequential(
    (0): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (stages): Sequential(
    (0): MetaNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): MetaNeXtBlock(
          (token_mixer): InceptionDWConv2d(
            (dwconv_hw): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=12)
            (dwconv_w): Conv2d(12, 12, kernel_size=(1, 11), stride=(1, 1), padding=(0, 5), groups=12)
            (dwconv_h): Conv2d(12, 12, kernel_size=(11, 1), stride=(1, 1), padding=(5, 0), groups=12)
          )
          (norm): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (mlp): ConvMlp(
            (fc1): Conv2d(96, 384, kernel_size=(1, 1), stride=(1, 1))
            (norm): Identity()
            (act): GELU(approximate='none')
            (drop): Dropout(p=0.0, inplace=False)
 

In [24]:
import torch
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

total_samples = 0
correct_predictions = 0
predicted_labels = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = x(images)
        predicted = (outputs > 0.5).float()
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = correct_predictions / total_samples

f1 = f1_score(true_labels, predicted_labels)

In [26]:
f1

0.33954451345755693